In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
import kerastuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [6]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [9]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics = ['accuracy'])
    
    return model

In [10]:
tuner = kt.Hyperband(model_builder, 
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [11]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [12]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f'''
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
''')

Trial 30 Complete [00h 00m 28s]
val_accuracy: 0.8701000213623047

Best val_accuracy So Far: 0.8884999752044678
Total elapsed time: 00h 05m 34s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 320 and the optimal learning rate for the optimizer
is 0.001.



In [13]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4802 - accuracy: 0.8293 - val_loss: 0.4054 - val_accuracy: 0.8572
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3639 - accuracy: 0.8674 - val_loss: 0.3657 - val_accuracy: 0.8659
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3275 - accuracy: 0.8802 - val_loss: 0.3557 - val_accuracy: 0.8711
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3015 - accuracy: 0.8896 - val_loss: 0.3359 - val_accuracy: 0.8777
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2835 - accuracy: 0.8951 - val_loss: 0.3531 - val_accuracy: 0.8748
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2669 - accuracy: 0.8990 - val_loss: 0.3446 - val_accuracy: 0.8765
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2549 - accuracy: 0.9046 - val_loss: 0.3376 - val_accuracy: